Извлечение имен из текста.
    
Параметры:
    content (str): Входной текст
    
Возвращает:
    set: Все извлеченные имена

In [ ]:
from yargy import Parser, rule
from yargy.predicates import gram

def parse_names(content):

    NAME = rule(
        gram('Name'), 
        gram('Surn') 
    )

    name_parser = Parser(NAME)

    names = set()
    for match in name_parser.findall(content):
        name = " ".join(token.value for token in match.tokens)
        names.add(name)

    return names

Извлечение географических названий из текста.
    
Параметры:
    content (str): Входной текст
    
Возвращает:
    set: Все извлеченные географические названия

In [ ]:
def parse_locations(content):

    LOCATION = rule(
        gram('Geox')
    )

    location_parser = Parser(LOCATION)

    locations = set()
    for match in location_parser.findall(content):
        location = " ".join(token.value for token in match.tokens)
        locations.add(location)

    return locations

Извлечение дат из текста.
    
Параметры:
    content (str): Входной текст
    
Возвращает:
    set: Все извлеченные даты

In [ ]:
from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline
from yargy.predicates import eq, custom

def parse_dates(content):

    def is_integer(value):
        return value.isdigit() and 1 <= len(value) <= 4

    INTEGER = custom(is_integer)

    MONTHS = morph_pipeline([
        'января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
        'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'
    ])

    DATE_WITH_YEAR = rule(
        INTEGER,
        MONTHS,
        INTEGER.optional(),
        eq('года').optional()
    )

    DATE_SHORT = rule(
        INTEGER,
        MONTHS
    )

    PREFIX_DATE = rule(
        or_(
            eq('в'), eq('с'), eq('до'), eq('на'), eq('после')
        ),
        or_(
            DATE_WITH_YEAR,
            DATE_SHORT
        )
    )

    GENERAL_DATE = or_(
        PREFIX_DATE,
        DATE_WITH_YEAR,
        DATE_SHORT
    )

    parser = Parser(GENERAL_DATE)

    dates = set()
    for match in parser.findall(content):
        date = " ".join(token.value for token in match.tokens)
        dates.add(date)
    return dates


Построчное чтение файла с извлечением имен, географических названий и дат.
    
Параметры:
    file_path (str): Путь к файлу

In [ ]:
def process_file(file_path):

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:

            if not line.strip():
                continue
            
            parts = line.strip().split('\t')
            if len(parts) < 3:
                continue

            rubric, headline, text = parts
            content = f"{headline} {text}"

            names = parse_names(content)
            if not names:
                continue

            locations = parse_locations(content)

            dates = parse_dates(content)

            print(f"Рубрика: {rubric}")
            print(f"Заголовок: {headline}")
            print(f"Имена: {', '.join(names)}")
            print(f"Места: {', '.join(locations) if locations else 'Не найдено'}")
            print(f"Даты: {', '.join(dates) if dates else 'Не найдено'}")
            print()


In [28]:
file_path = '../data/news.txt'
process_file(file_path)


Рубрика: sport
Заголовок: Матс Сундин стал советником тренера сборной Швеции по хоккею
Имена: Матс Сундин
Места: Швеции, Турине, Торонто, Сочи, Ванкувер, Квебек, Финляндии
Даты: 12 ноября 2012 года

Рубрика: media
Заголовок: Брендом года по версии EFFIE впервые стал город
Имена: Михаилом Прохоровым, Владимир Филиппов, Ильгизу Фахриеву
Места: Казани, Казань, мобиль, России, Владимир
Даты: 12 апреля

Рубрика: culture
Заголовок: Стинг выставит накопленное имущество на аукцион
Имена: Бена Николсона, Пабло Пикассо, Анри Матисса, Жоржа Брака
Места: Накасимы, принты, Вестминстере
Даты: 24 февраля 2016

Рубрика: sport
Заголовок: Бывший тренер «Ростова» рассказал о воспитании сербов в любви к России
Имена: Владимиром Путиным
Места: Запад, Ливия, Амкар, Россия, Ростов, Америка, Сербии, Ростова, Европа, Владимиром, Черногории, Москва, России, Црвену, Црвены, Сирия, Ирак
Даты: 1 июня 2015 года

Рубрика: forces
Заголовок: «Адмирал Кузнецов» и «Петр Великий» отправились в дальнее плавание
Имена: Лео